In [3]:
import torch

pt_file = torch.load("./geometric_data_processed org.pt")

pt_file
# x_index_slice = pt_file[1]['x']
# edge_index = pt_file[0].edge_index
# edge_attr = pt_file[0].edge_attr
# edge_index_slice = pt_file[1]['edge_index']

(Data(edge_index=[2, 4773974], edge_attr=[4773974, 3], x=[2344859, 9], y=[15535, 10]),
 defaultdict(dict,
             {'edge_index': tensor([      0,     244,     926,  ..., 4773604, 4773842, 4773974]),
              'edge_attr': tensor([      0,     244,     926,  ..., 4773604, 4773842, 4773974]),
              'x': tensor([      0,     119,     457,  ..., 2344679, 2344792, 2344859]),
              'y': tensor([    0,     1,     2,  ..., 15533, 15534, 15535])}))

In [23]:
import torch
from tqdm import tqdm

pt_file = torch.load("./geometric_data_processed org.pt")

x = pt_file[0].x
y = pt_file[0].y
x_index_slice = pt_file[1]['x']
edge_index = pt_file[0].edge_index
edge_attr = pt_file[0].edge_attr
edge_index_slice = pt_file[1]['edge_index']
numberOfGraphs = pt_file[0].y.shape[0]

# line graph node
# lg_x = edge_attr
lg_x = []
lg_x_slice = edge_index_slice

# line graph edge
lg_edge = []
lg_edge_attr = []
lg_edge_slice = [0]


# graphIdx = 0
for graphIdx in tqdm(range(numberOfGraphs)):
  # print()
  graphX = x[x_index_slice[graphIdx]:x_index_slice[graphIdx+1], :]

  linegraphNodeAttr_edge = edge_attr[edge_index_slice[graphIdx]:edge_index_slice[graphIdx+1], :]
  linegraphNodeAttr_node = graphX[edge_index[:, edge_index_slice[graphIdx]:edge_index_slice[graphIdx+1]].T]
  # linegraphNodeAttr_node = torch.cat([, linegraphNodeAttr_node[:, 1, :]], dim=1)
  # print(lg_x[lg_x_slice[graphIdx]:lg_x_slice[graphIdx+1], :].shape)
  # print(linegraphNodeAttr_node.shape)

  graphEdgeIdx = edge_index[:, edge_index_slice[graphIdx]:edge_index_slice[graphIdx+1]]
  graphEdgeAttr = edge_attr[edge_index_slice[graphIdx]:edge_index_slice[graphIdx+1], :]


  # NOTE: Add global virtual node to graph
  globalVirtualNodeX = torch.zeros(1, graphX.shape[1])
  # newNodeNumber = graphX.shape[0]
  newNodeNumber = max(graphEdgeIdx[0, :]) + 1
  graphX = torch.cat([graphX, globalVirtualNodeX], dim=0)

  uniqueNodes = torch.unique(graphEdgeIdx[0,:])
  newEdge1 = torch.tensor([[i, newNodeNumber] for i in range(graphX.shape[0]-1)]).T
  newEdge2 = torch.tensor([[newNodeNumber, i] for i in range(graphX.shape[0]-1)]).T
  # print(uniqueNodes.shape)
  # print(uniqueNodes)
  # print(newEdge1.shape)
  # print(newEdge1)
  # print(newEdge1[:, -4:])
  graphEdgeIdx = torch.cat([graphEdgeIdx, newEdge1, newEdge2], dim=1)
  graphEdgeAttr = torch.cat([graphEdgeAttr, torch.zeros([newEdge1.shape[1]*2, graphEdgeAttr.shape[1]])], dim=0)

  # NOTE: Making line graph edges
  linegraphNodeIdx = graphEdgeIdx.T
  linegraphEdgeIdxMask = torch.nonzero(
    (linegraphNodeIdx[:, 1, None] == linegraphNodeIdx[:, 0]) &
    (linegraphNodeIdx[:, 0, None] != linegraphNodeIdx[:, 1])
  )
  linegraphEdgeIdx = linegraphNodeIdx[linegraphEdgeIdxMask]
  linegraphEdgeAttr_node = graphX[linegraphEdgeIdx[:, 0, 1]]

  # NOTE: line graph edge attribute
  edgeStartMask = torch.all(graphEdgeIdx.T.unsqueeze(0) == edgeStartIdx.unsqueeze(1) , dim=2).nonzero(as_tuple=False)[:,1]
  edgeEndMask = torch.all(graphEdgeIdx.T.unsqueeze(0) == edgeEndIdx.unsqueeze(1), dim=2).nonzero(as_tuple=False)[:,1]
  edgeStartAttr = graphEdgeAttr[edgeStartMask]
  edgeEndAttr = graphEdgeAttr[edgeEndMask]
  # print(linegraphEdgeIdx.shape)
  # print(edgeStartIdx.shape)
  # print(edgeStartAttr.shape)


  lg_x.append(torch.cat([linegraphNodeAttr_edge, linegraphNodeAttr_node[:, 0, :], linegraphNodeAttr_node[:, 1, :]], dim=1))
  lg_edge.append(linegraphEdgeIdxMask.T)
  lg_edge_attr.append(linegraphEdgeAttr)
  lg_edge_slice.append(lg_edge_slice[-1]+linegraphEdgeIdxMask.shape[0])



  



100%|██████████| 15535/15535 [14:49<00:00, 17.47it/s]


In [25]:
print(lg_x[0].shape)
print(lg_edge[0].shape)
print(lg_edge_attr[0].shape)
print(len(lg_edge_slice))

torch.Size([244, 21])
torch.Size([2, 14858])
torch.Size([15334, 15])
15536


In [26]:
# NOTE: Create tensors for line graph edge, edge_attr, edge_slice
lg_x = torch.cat(lg_x, dim=0)
lg_edge = torch.cat(lg_edge, dim=1)
lg_edge_attr = torch.cat(lg_edge_attr, dim=0)
lg_edge_slice = torch.tensor(lg_edge_slice, dtype=torch.int64)

# NOTE: create line graph dataset
pt_file = torch.load("./geometric_data_processed copy.pt")

pt_file[0].x = lg_x
pt_file[1]['x'] = lg_x_slice
pt_file[0].edge_index = lg_edge
pt_file[0].edge_attr = lg_edge_attr
pt_file[1]['edge_index'] = lg_edge_slice
pt_file[1]['edge_attr'] = lg_edge_slice

torch.save(pt_file, "./geometric_data_processed lg-gvn.pt")